# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 17 2022 12:00AM,239807,10,8153649,Eywa Pharma Inc.,Released
1,Jun 17 2022 12:00AM,239807,10,8153650,Eywa Pharma Inc.,Released
2,Jun 17 2022 12:00AM,239807,10,8153653,Eywa Pharma Inc.,Released
3,Jun 17 2022 12:00AM,239807,10,8153654,Eywa Pharma Inc.,Released
4,Jun 17 2022 12:00AM,239807,10,8153655,Eywa Pharma Inc.,Released
5,Jun 17 2022 12:00AM,239807,10,6594549,Eywa Pharma Inc.,Released
6,Jun 17 2022 12:00AM,239802,10,SO94288,"Senseonics, Incorporated",Released
7,Jun 16 2022 4:19PM,239805,16,6594530,"SD Head USA, LLC",Released
8,Jun 16 2022 4:03PM,239801,16,SHL-8153621,"SHL Pharma, LLC",Released
9,Jun 16 2022 2:46PM,239794,10,CTF0011273,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,239794,Released,1
27,239801,Released,1
28,239802,Released,1
29,239805,Released,1
30,239807,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239794,NaN,NaN,1.0
239801,NaN,NaN,1.0
239802,NaN,NaN,1.0
239805,NaN,NaN,1.0
239807,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239692,0.0,0.0,1.0
239694,0.0,0.0,1.0
239699,0.0,0.0,1.0
239717,14.0,0.0,3.0
239720,0.0,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239692,0,0,1
239694,0,0,1
239699,0,0,1
239717,14,0,3
239720,0,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239692,0,0,1
1,239694,0,0,1
2,239699,0,0,1
3,239717,14,0,3
4,239720,0,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239692,,,1
1,239694,,,1
2,239699,,,1
3,239717,14,,3
4,239720,,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 17 2022 12:00AM,239807,10,Eywa Pharma Inc.
6,Jun 17 2022 12:00AM,239802,10,"Senseonics, Incorporated"
7,Jun 16 2022 4:19PM,239805,16,"SD Head USA, LLC"
8,Jun 16 2022 4:03PM,239801,16,"SHL Pharma, LLC"
9,Jun 16 2022 2:46PM,239794,10,"Citieffe, Inc."
10,Jun 16 2022 2:40PM,239792,19,ACG North America LLC
11,Jun 16 2022 2:24PM,239790,21,"NBTY Global, Inc."
12,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions
19,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals
20,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 17 2022 12:00AM,239807,10,Eywa Pharma Inc.,,,6
1,Jun 17 2022 12:00AM,239802,10,"Senseonics, Incorporated",,,1
2,Jun 16 2022 4:19PM,239805,16,"SD Head USA, LLC",,,1
3,Jun 16 2022 4:03PM,239801,16,"SHL Pharma, LLC",,,1
4,Jun 16 2022 2:46PM,239794,10,"Citieffe, Inc.",,,1
5,Jun 16 2022 2:40PM,239792,19,ACG North America LLC,,,1
6,Jun 16 2022 2:24PM,239790,21,"NBTY Global, Inc.",,1,
7,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions,,,7
8,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals,,,1
9,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 12:00AM,239807,10,Eywa Pharma Inc.,6,,
1,Jun 17 2022 12:00AM,239802,10,"Senseonics, Incorporated",1,,
2,Jun 16 2022 4:19PM,239805,16,"SD Head USA, LLC",1,,
3,Jun 16 2022 4:03PM,239801,16,"SHL Pharma, LLC",1,,
4,Jun 16 2022 2:46PM,239794,10,"Citieffe, Inc.",1,,
5,Jun 16 2022 2:40PM,239792,19,ACG North America LLC,1,,
6,Jun 16 2022 2:24PM,239790,21,"NBTY Global, Inc.",,1,
7,Jun 16 2022 2:18PM,239787,16,Sartorius Bioprocess Solutions,7,,
8,Jun 16 2022 2:14PM,239786,10,Amatheon Pharmaceuticals,1,,
9,Jun 16 2022 2:02PM,239781,10,Acute Outpatient Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 12:00AM,239807,10,Eywa Pharma Inc.,6,,
1,Jun 17 2022 12:00AM,239802,10,"Senseonics, Incorporated",1,,
2,Jun 16 2022 4:19PM,239805,16,"SD Head USA, LLC",1,,
3,Jun 16 2022 4:03PM,239801,16,"SHL Pharma, LLC",1,,
4,Jun 16 2022 2:46PM,239794,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 12:00AM,239807,10,Eywa Pharma Inc.,6.0,NaN,NaN
1,Jun 17 2022 12:00AM,239802,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jun 16 2022 4:19PM,239805,16,"SD Head USA, LLC",1.0,NaN,NaN
3,Jun 16 2022 4:03PM,239801,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Jun 16 2022 2:46PM,239794,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 12:00AM,239807,10,Eywa Pharma Inc.,6.0,0.0,0.0
1,Jun 17 2022 12:00AM,239802,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jun 16 2022 4:19PM,239805,16,"SD Head USA, LLC",1.0,0.0,0.0
3,Jun 16 2022 4:03PM,239801,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Jun 16 2022 2:46PM,239794,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2157862,16.0,10.0,1.0
15,479525,20.0,0.0,1.0
16,1918126,14.0,0.0,0.0
19,479544,2.0,0.0,0.0
20,958933,24.0,0.0,14.0
21,239790,0.0,1.0,0.0
22,239753,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2157862,16.0,10.0,1.0
1,15,479525,20.0,0.0,1.0
2,16,1918126,14.0,0.0,0.0
3,19,479544,2.0,0.0,0.0
4,20,958933,24.0,0.0,14.0
5,21,239790,0.0,1.0,0.0
6,22,239753,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,16.0,10.0,1.0
1,15,20.0,0.0,1.0
2,16,14.0,0.0,0.0
3,19,2.0,0.0,0.0
4,20,24.0,0.0,14.0
5,21,0.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,15,Released,20.0
2,16,Released,14.0
3,19,Released,2.0
4,20,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,1.0,1.0,0.0,0.0,14.0,0.0,0.0
Executing,10.0,0.0,0.0,0.0,0.0,1.0,0.0
Released,16.0,20.0,14.0,2.0,24.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,1.0,1.0,0.0,0.0,14.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Released,16.0,20.0,14.0,2.0,24.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,1.0,1.0,0.0,0.0,14.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Released,16.0,20.0,14.0,2.0,24.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()